In [ ]:
# ! pip3 cache purge
# ! pip3 install -r requirements.txt --no-cache-dir zfit

In [ ]:
# ! pip install --upgrade pip
! pip install --user zfit
# ! pip3 install -r requirements.txt --no-cache-dir zfit

In [ ]:
import uproot
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import mplhep
import zfit
from zfit import z
import tensorflow as tf

In [ ]:
sys.path.append("/home/belle2/amubarak/Ds2D0enue_Analysis/11-Python_Functions/")

In [ ]:
from Functions import plot_model

# Import Data

In [ ]:
# In this notebook we only process the main signal and the generic events,
# for illustration purposes.
# You can add other backgrounds after if you wish.
samples = ["Signal", "ccbar"]

In [ ]:
DataFrames = {}  # define empty dictionary to hold dataframes
Date = "1112"
Attempt = "0"

# Signal:
DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree",library='pd')
# Background
for s in samples[1:]: # loop over samples
    DataFrames[s] =  uproot.concatenate("/home/belle2/amubarak/C03-Grid/Completed/Ds2D0e-Generic_Ds_" + Date +"24_"+ Attempt +"_"+ s +".root:Dstree",library='pd')

The line below is to look at the available variables.

In [ ]:
print(DataFrames["Signal"].columns.tolist())

### Setup
The code below will be used to apply cuts to the data.

In [ ]:
# Electron ID
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['e_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['e_electronID']>=0.95]
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_em_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_em_electronID']>=0.95]

# Photon Conversion
#-------------------
DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=0.1]
DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=0.1]

# Peaking Background Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_diff_D0pi']>=0.15)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.15)]

# # Vertex Fitting
# #----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb']>=0.01]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_chiProb']>=0.01]

# Dalitz Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["ccbar"]['Ds_pi0veto_M_Correction']>=0.16)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["Signal"]['Ds_pi0veto_M_Correction']>=0.16)]

# Vertex Fit
#----------------
DataFrames["Signal"] = DataFrames["Signal"][DataFrames[samples[0]]['Ds_chiProb_rank']==1]
DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames[samples[1]]['Ds_chiProb_rank']==1]

# # D0 Invariant Mass
# #-----------------------
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['D0_dM']>=-0.02) & (DataFrames["ccbar"]['D0_dM']<=0.02)]
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['D0_dM']>=-0.02) & (DataFrames["ccbar"]['D0_dM']<=0.02)]

# Build Model

### Parameters

In [ ]:
sig_mean = zfit.Parameter("sig_mean", 3.6079e-04, -0.005, 0.005)
sig_sigma = zfit.Parameter("sig_sigma", 4.7124e-03, 1e-4, 0.02)
sig_alphaL = zfit.Parameter("sig_alphaL", 1.8118e+00, 1e-4, 4.)
sig_alphaR = zfit.Parameter("sig_alphaR", 1.4095e+00, 1e-4, 4.)
sig_nL = zfit.Parameter("sig_nL", 7.4048e-01, 1e-4, 100)
sig_nR = zfit.Parameter("sig_nR", 3.0364e+00, 1e-4, 100)

c0 = zfit.Parameter("c0", 0, -1, 1)

### Model

In [ ]:
D0_dM = (-0.06,0.06)
obs = zfit.Space('D0_dM', D0_dM)

signal_pdf = zfit.pdf.DoubleCB(mu=sig_mean, sigma=sig_sigma, alphal=sig_alphaL, nl=sig_nL, alphar=sig_alphaR, nr=sig_nR, obs=obs)
comb_bkg_pdf = zfit.pdf.Chebyshev(obs=obs, coeffs=[], apply_scaling=True, coeff0=True, name='Chebyshev')

sig_yield = zfit.Parameter('sig_yield', 282350, 100000., 600000, step_size=1)  # step size: default is small, use appropriate
bkg_yield = zfit.Parameter('bkg_yield', 32687, 0., 100000, step_size=1)

# Create the extended models
extended_sig = signal_pdf.create_extended(sig_yield)
extended_bkg = comb_bkg_pdf.create_extended(bkg_yield)

# The final model is the combination of the signal and backgrond PDF
model = zfit.pdf.SumPDF([extended_bkg, extended_sig])

In [ ]:
# Fill an array with the generic MC data to be fit
data_np = DataFrames["ccbar"].D0_dM.to_numpy()
data = zfit.data.Data.from_numpy(obs=obs, array=data_np)
data.set_data_range(obs)
data_size = data.n_events

In [ ]:
# Simultaenous loss
nll_simultaneous = zfit.loss.ExtendedUnbinnedNLL(model=[model], data=[data])
minimizer = zfit.minimize.Minuit()
result = minimizer.minimize(nll_simultaneous)

In [ ]:
result.hesse()
errors, new_result = result.errors()
print(result)

In [ ]:
plot_model(model=model, mydata=data, myrange=[-0.05,0.05], nevents=int(data_size), nbins=50)

In [ ]:
# plot_model(model=model, mydata=data, nevents=int(data_size), nbins=50, myrange=obs, mylabel="", mytitle = "", plot_data=True)